# Calculate distance from NHP

This notebook helps calculate the distance from a patient's residental area to National Hospital of Pediatrics, Hanoi, Vietnam, using `Azure Maps Route` and `Geopy` libraries.

In [1]:
# Import necessary libraries
from azure.maps.route import MapsRouteClient
from azure.core.credentials import AzureKeyCredential
from azure.maps.route.models import LatLon
from geopy.geocoders import AzureMaps
from pprint import pprint
import pandas as pd

In [ ]:
# Create map and route clients
credential = AzureKeyCredential('AZURE_MAPS_KEY')
geolocation_client = AzureMaps(credential.key)
route_client = MapsRouteClient(credential=credential)

In [ ]:
# Set default location for NHP to prevent miscalculation
NHP_coordinates = LatLon(21.026047, 105.810077)
print('NHP coordinates:', geolocation_client.reverse("%f, %f" % (NHP_coordinates.lat, NHP_coordinates.lon)))

In [ ]:
# Get data from a csv file using pandas
csv_df = pd.read_csv('../test.csv')
addrs = list(map(geolocation_client.geocode, csv_df['complete_patient_address']))

In [ ]:
# Get distance from NHP to each address using Azure Maps Route API
def get_distance(addr):
    route = route_client.get_route_directions(route_points=[
        LatLon(addr.latitude, addr.longitude),
        NHP_coordinates
    ])
    return route.as_dict()['routes'][0]['summary']

In [ ]:
distances = []

for addr in addrs:
    dist = get_distance(addr)
    distances.append(dist)

    # print('Distance between %s (%f,%f) and NHP(%f,%f): %skm in %s'
    #       % (addr, addr.latitude, addr.longitude, NHP_coordinates.lat, NHP_coordinates.lon,
    #          dist['length_in_meters'] / 1000, str(timedelta(seconds=dist['travel_time_in_seconds']))))
    
print("Finish calculating distances")

In [ ]:
# Remove old column
if 'patient_home_coordinate' in csv_df.keys():
    csv_df.drop('patient_home_coordinate', axis=1, inplace=True)
csv_df.insert(0, 'patient_home_coordinate', list(map(lambda addr: "%f,%f" % (addr.latitude, addr.longitude), addrs)))

# Remove old column
if 'distance_to_nhp' in csv_df.keys():
    csv_df.drop('distance_to_nhp', axis=1, inplace=True)
csv_df.insert(0, 'distance_to_nhp', list(map(lambda d: d['length_in_meters'] / 1000, distances)))

In [ ]:
# Export to csv file
csv_df.to_csv('../export.csv', index=False)

# Histogram

In [ ]:
import matplotlib.pyplot as plt